Projeto Sistemas Inteligentes

Grupo
    Gustavo Alves de Lima RA: 11103516
    Henrique Soares Silva RA: 11071116
    Jessica Faria Rodrigues Zeferino RA: 11054416
    Lucas Garcia Fracaro RA: 11026316
    Marcelo Tranche Junior RA:11201810416
    

In [1]:
!pip install tensorflow

In [2]:
#importação das bibliotecas necessárias para a construção do modelo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras
import tensorflow as tf
from sklearn.model_selection import train_test_split


2021-12-01 22:55:44.912665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-01 22:55:44.912702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
#leitura dos dados e criação do dataset

data = pd.read_csv("age_gender.csv")
print(data.shape)
data.head

(23705, 5)


<bound method NDFrame.head of        age  ethnicity  gender                        img_name  \
0        1          2       0  20161219203650636.jpg.chip.jpg   
1        1          2       0  20161219222752047.jpg.chip.jpg   
2        1          2       0  20161219222832191.jpg.chip.jpg   
3        1          2       0  20161220144911423.jpg.chip.jpg   
4        1          2       0  20161220144914327.jpg.chip.jpg   
...    ...        ...     ...                             ...   
23700   99          0       1  20170120221920654.jpg.chip.jpg   
23701   99          1       1  20170120134639935.jpg.chip.jpg   
23702   99          2       1  20170110182418864.jpg.chip.jpg   
23703   99          2       1  20170117195405372.jpg.chip.jpg   
23704   99          0       1  20170110182052119.jpg.chip.jpg   

                                                  pixels  
0      129 128 128 126 127 130 133 135 139 142 145 14...  
1      164 74 111 168 169 171 175 182 184 188 193 199...  
2      67 70

In [4]:
#eliminando variável que não importa (nome da imagem)

data = data.drop('img_name', axis=1)
print(data.shape)
data.head

(23705, 4)


<bound method NDFrame.head of        age  ethnicity  gender  \
0        1          2       0   
1        1          2       0   
2        1          2       0   
3        1          2       0   
4        1          2       0   
...    ...        ...     ...   
23700   99          0       1   
23701   99          1       1   
23702   99          2       1   
23703   99          2       1   
23704   99          0       1   

                                                  pixels  
0      129 128 128 126 127 130 133 135 139 142 145 14...  
1      164 74 111 168 169 171 175 182 184 188 193 199...  
2      67 70 71 70 69 67 70 79 90 103 116 132 145 155...  
3      193 197 198 200 199 200 202 203 204 205 208 21...  
4      202 205 209 210 209 209 210 211 212 214 218 21...  
...                                                  ...  
23700  127 100 94 81 77 77 74 99 102 98 128 145 160 1...  
23701  23 28 32 35 42 47 68 85 98 103 113 117 130 129...  
23702  59 50 37 40 34 19 30 101 156 170 17

In [5]:
#separação dos dados para treinamento e teste

#coletando somente as informações sobre os pixels
data_pixel = data['pixels']
data_pixel.shape

(23705,)

In [6]:
#montando as dimenções da foto e guardando em forma de vetor

data_fotos = []

for i in range(data_pixel.shape[0]):
    data_fotos.append(np.array(data_pixel[i].split(' '), dtype='int64').reshape(48,48))

#uma amostra

data_fotos = np.array(data_fotos)

data_fotos[np.random.randint(0,100)]
    

array([[184, 183, 189, ...,  53,  62,  73],
       [184, 184, 182, ..., 140, 142, 145],
       [195, 190, 173, ..., 215, 191, 160],
       ...,
       [239, 239, 238, ..., 197, 204, 229],
       [238, 238, 239, ..., 201, 205, 228],
       [240, 240, 239, ..., 205, 203, 227]])

In [7]:
#converto variaveis categoricas em flags

gender = pd.get_dummies(data['gender']).values
ethnicity = pd.get_dummies(data['ethnicity']).values
age = data['age'].values
#separação entre os dados de treino e teste de 50%

#jogar um valor fora para conseguir treinar o modelo


#x_train, x_test, y_train_gender, y_test_gender, y_train_ethnicity, y_test_ethnicity, y_train_age, y_test_age = train_test_split(
    #data_pixel, gender, ethnicity, data['age'].values,test_size=0.5, random_state=1234)
    
X_train_ethnicity, X_test_ethnicity, y_train_ethnicity, y_test_ethnicity = train_test_split(data_fotos, ethnicity, test_size= 0.3)
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(data_fotos, gender, test_size= 0.3)
X_train_age, X_test_age, y_train_age, y_test_age = train_test_split(data_fotos, age, test_size= 0.3)

In [8]:
#importações para a criação do modelo

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling2D
#rom tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import clone_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, InputLayer

import keras
import IPython

In [9]:
#definindo r²

from keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [10]:
#definição do modelo

def modelo(num_classes, activation, loss, metric):
    modelo = Sequential() 
    modelo.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
    modelo.add(BatchNormalization())
    modelo.add(MaxPool2D(pool_size=(2, 2)))
    modelo.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))# padding = "same"))
    modelo.add(MaxPool2D(pool_size=(2, 2)))
    modelo.add(Flatten())
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(num_classes, activation= activation))
    modelo.compile(optimizer='Adam',
              loss= loss,
              metrics=[metric])
    return modelo

In [11]:
#plot_model(modelo(5,"softmax",'categorical_crossentropy'))

In [12]:
#!pip install graphviz

In [14]:
#200 gerações para treino com o tamanho de 64 fotos

#temos 5 categorias de etinia no dataset

early_stopping = EarlyStopping(patience=10, min_delta=0.001,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

geracoes = 100
tamanho = 32

#etnia
modelo_ethnicity = modelo(5,"softmax",'categorical_crossentropy', 'accuracy')
history_ethnicity = modelo_ethnicity.fit(X_train_ethnicity, y_train_ethnicity, batch_size=tamanho,
                              epochs = geracoes, validation_data = (X_test_ethnicity,y_test_ethnicity), steps_per_epoch= X_train_ethnicity.shape[0] // tamanho, callbacks= [early_stopping, learning_rate_reduction])

#genero

modelo_gender = modelo(2,"softmax",'categorical_crossentropy', 'accuracy')
history_gender = modelo_gender.fit(X_train_gender, y_train_gender, batch_size=tamanho,
                              epochs = geracoes, validation_data = (X_test_gender,y_test_gender), steps_per_epoch= X_train_gender.shape[0] // tamanho, callbacks= [early_stopping, learning_rate_reduction])
#idade

modelo_age = modelo(1,"relu",'mean_squared_error', coeff_determination )
history_age = modelo_age.fit(X_train_age, y_train_age, batch_size=tamanho,
                              epochs = geracoes, validation_data = (X_test_age,y_test_age), steps_per_epoch= X_train_age.shape[0] // tamanho, callbacks= [early_stopping, learning_rate_reduction])


2021-12-01 23:15:33.550311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 305842176 exceeds 10% of free system memory.


Epoch 1/100
518/518 [==============================] - 23s 43ms/step - loss: 1.3341 - accuracy: 0.4928 - val_loss: 1.0343 - val_accuracy: 0.5679 - lr: 0.0010
Epoch 2/100
518/518 [==============================] - 22s 42ms/step - loss: 1.1383 - accuracy: 0.5444 - val_loss: 1.0524 - val_accuracy: 0.5775 - lr: 0.0010
Epoch 3/100
518/518 [==============================] - 22s 42ms/step - loss: 1.0814 - accuracy: 0.5713 - val_loss: 0.9769 - val_accuracy: 0.6583 - lr: 0.0010
Epoch 4/100
518/518 [==============================] - 22s 42ms/step - loss: 1.0415 - accuracy: 0.5919 - val_loss: 0.8725 - val_accuracy: 0.6895 - lr: 0.0010
Epoch 5/100
518/518 [==============================] - 22s 42ms/step - loss: 1.0074 - accuracy: 0.6036 - val_loss: 0.8457 - val_accuracy: 0.6978 - lr: 0.0010
Epoch 6/100
518/518 [==============================] - 22s 42ms/step - loss: 0.9863 - accuracy: 0.6134 - val_loss: 0.8477 - val_accuracy: 0.6945 - lr: 0.0010
Epoch 7/100
518/518 [==============================]

518/518 [==============================] - 22s 42ms/step - loss: 0.8092 - accuracy: 0.6680 - val_loss: 0.7590 - val_accuracy: 0.7458 - lr: 0.0010
Epoch 22/100
518/518 [==============================] - 21s 40ms/step - loss: 0.7776 - accuracy: 0.6898 - val_loss: 0.7327 - val_accuracy: 0.7507 - lr: 0.0010
Epoch 23/100
518/518 [==============================] - 21s 40ms/step - loss: 0.7678 - accuracy: 0.6949 - val_loss: 0.7872 - val_accuracy: 0.7388 - lr: 0.0010
Epoch 24/100
518/518 [==============================] - 21s 40ms/step - loss: 0.7557 - accuracy: 0.6983 - val_loss: 0.7461 - val_accuracy: 0.7555 - lr: 0.0010
Epoch 25/100
518/518 [==============================] - 21s 40ms/step - loss: 0.7434 - accuracy: 0.7018 - val_loss: 0.8026 - val_accuracy: 0.7192 - lr: 0.0010
Epoch 26/100
518/518 [==============================] - 21s 40ms/step - loss: 0.7344 - accuracy: 0.7045 - val_loss: 0.7649 - val_accuracy: 0.7532 - lr: 0.0010
Epoch 27/100
518/518 [==============================] - 21s

2021-12-01 23:25:40.771354: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 305842176 exceeds 10% of free system memory.


Epoch 1/100
518/518 [==============================] - 22s 41ms/step - loss: 0.4804 - accuracy: 0.7747 - val_loss: 0.3584 - val_accuracy: 0.8351 - lr: 0.0010
Epoch 2/100
518/518 [==============================] - 22s 42ms/step - loss: 0.3391 - accuracy: 0.8462 - val_loss: 0.3073 - val_accuracy: 0.8594 - lr: 0.0010
Epoch 3/100
518/518 [==============================] - 21s 41ms/step - loss: 0.3048 - accuracy: 0.8658 - val_loss: 0.2968 - val_accuracy: 0.8645 - lr: 0.0010
Epoch 4/100
518/518 [==============================] - 21s 40ms/step - loss: 0.2726 - accuracy: 0.8797 - val_loss: 0.2795 - val_accuracy: 0.8757 - lr: 0.0010
Epoch 5/100
518/518 [==============================] - 21s 40ms/step - loss: 0.2629 - accuracy: 0.8807 - val_loss: 0.2733 - val_accuracy: 0.8810 - lr: 0.0010
Epoch 6/100
518/518 [==============================] - 21s 40ms/step - loss: 0.2454 - accuracy: 0.8903 - val_loss: 0.2634 - val_accuracy: 0.8886 - lr: 0.0010
Epoch 7/100
518/518 [==============================]

518/518 [==============================] - 21s 41ms/step - loss: 157.3248 - coeff_determination: 0.5712 - val_loss: 144.8432 - val_coeff_determination: 0.5971 - lr: 0.0010
Epoch 5/100
518/518 [==============================] - 21s 40ms/step - loss: 146.5412 - coeff_determination: 0.5972 - val_loss: 109.9315 - val_coeff_determination: 0.7002 - lr: 0.0010
Epoch 6/100
518/518 [==============================] - 21s 40ms/step - loss: 143.6185 - coeff_determination: 0.6076 - val_loss: 106.8243 - val_coeff_determination: 0.7121 - lr: 0.0010
Epoch 7/100
518/518 [==============================] - 21s 40ms/step - loss: 136.5939 - coeff_determination: 0.6254 - val_loss: 101.2177 - val_coeff_determination: 0.7268 - lr: 0.0010
Epoch 8/100
518/518 [==============================] - 21s 40ms/step - loss: 137.1379 - coeff_determination: 0.6239 - val_loss: 103.9720 - val_coeff_determination: 0.7187 - lr: 0.0010
Epoch 9/100
518/518 [==============================] - 21s 40ms/step - loss: 131.5315 - coef

In [ ]:
!pip install pickle-mixin

In [15]:
import pickle

#salvar o modelo
filehandler = open("model_ethnicity.obj","wb")
pickle.dump(modelo_ethnicity,filehandler)

filehandler = open("model_gender.obj","wb")
pickle.dump(modelo_gender,filehandler)

filehandler = open("model_age.obj","wb")
pickle.dump(modelo_age,filehandler)

#load modelo

object_file = pickle.load(open("model_ethnicity.obj",'rb'))

2021-12-02 08:34:16.924824: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://427ee8f9-e5e4-44b1-9a32-60ffab4056c7/assets
INFO:tensorflow:Assets written to: ram://7eb4a2a1-3d08-4e16-bb99-7eca38724a79/assets
INFO:tensorflow:Assets written to: ram://e7626a2a-c1fe-4ab0-91d3-5e8a5d98920f/assets
